In [5]:
''' new version 090925 DJ
    Generate surface 1/2 or 1 voxel up and down from the surface
 '''
# Import required libraries
import numpy as np
import nibabel as nib
from scipy.ndimage import map_coordinates
import os
import matplotlib.pyplot as plt
import re
from scipy.stats import zscore

#method 0: half voxel up and down (if spacing_mm is 0.12mm, then 0.06mm up and down)
#method 1: full voxel length along the surface normal (if spacing_mm is 0.12mm, then 0.12mm up and down)
#090925 DJ - Ting suggested to keep the spacing_mm as 0.24mm or 0.12mm and maximum steps at 8 or 16, respectively

params = {'spacing_mm': 0.24, 'dist_max_mm': 0.96, 'method':0}
#get all the folders (subjects) in the current directory
path = "/Users/dennis.jungchildmind.org/Desktop/exvivo"
save_path = "/Users/dennis.jungchildmind.org/Desktop/exvivo_voxel_up_and_down"

subject_folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]


def get_surface_xyz(path_surf_norm, path_surf_coords, spacing_mm=0.12, dist_max_mm=2):
    """Generate and plot intensity differences at varying distances from cortical surface"""
    
    # Load surface data
    surf_norm = nib.load(path_surf_norm)
    surf_coords = nib.load(path_surf_coords)
    
    # Extract coordinates and normals
    norm_xyz = np.array([surf_norm.darrays[i].data for i in range(3)])
    surf_xyz = np.array([surf_coords.darrays[i].data for i in range(3)])

    #method 1 
    if params['method'] == 0:
        #calculate half voxel up (spacing mm/2) and down (spacing mm/2) from the surface along the surfarce normal 
        dist_array = np.flipud(np.concatenate([-np.arange(spacing_mm/2, dist_max_mm, spacing_mm)[::-1], np.arange(spacing_mm/2, dist_max_mm, spacing_mm)]))
    elif params['method'] == 1:
        #calculate full voxel length along the surface normal
        dist_array = np.flipud(np.concatenate([-np.arange(spacing_mm, dist_max_mm + spacing_mm, spacing_mm)[::-1], [0], 
                                        np.arange(spacing_mm, dist_max_mm + spacing_mm, spacing_mm)]))
    
    #calcualte the voxel coordinates up along the surface normal
    all_points = [surf_xyz.T + norm_xyz.T * d for d in dist_array]
    
    return all_points, dist_array


for subject_name in subject_folders:
    #check if there is a file that includes  "32k_fs_LR.surf.gii"
    #check as regex 
    for layers in ["inf"]:
        regex_target_1 = f"{layers}.32k_fs_LR.surfnorm.func.gii"
        regex_target_2 = f"{layers}.32k_fs_LR.coord.func.gii"

        files = os.listdir(os.path.join(path, subject_name))
        matching_surfnorm = [f for f in files if regex_target_1 in f]
        matching_coord = [f for f in files if regex_target_2 in f]
        print(matching_surfnorm)
        print(matching_coord)
        
        if matching_surfnorm and matching_coord:
            for i,file in enumerate(matching_surfnorm):
                surfacenorm_file = os.path.join(path, subject_name, matching_surfnorm[i])
                coord_file = os.path.join(path, subject_name, matching_coord[i])

                #get the hemisphere from the file name
                all_points, dist_array = get_surface_xyz(
                    surfacenorm_file,
                    coord_file,
                    spacing_mm=params['spacing_mm'],
                    dist_max_mm=params['dist_max_mm']
                )
               
                hemi = matching_surfnorm[0].split('.')[0].split('_')[-1]
                #subject_name = 'I48_new_confidence'
                tmp = nib.load(os.path.join(path, subject_name, f'{hemi}.inf.32k_fs_LR.surf.gii'))

                # Create output directory once
                output_dir = os.path.join(save_path, subject_name, f'surf_voxel_up_and_down', f'{int(params["spacing_mm"]*1000)}um_method{params["method"]}')
                os.makedirs(output_dir, exist_ok=True)

                for j in range(len(all_points)):
                    # Force assignment using setfield
                    new_data = np.array(all_points[j], dtype=tmp.darrays[0].data.dtype)
                    tmp.darrays[0].__dict__['data'] = new_data

                    output_file = os.path.join(output_dir, f'{hemi}.inf.32k_fs_LR_{dist_array[j]:.2f}.surf.gii')
                    nib.save(tmp, output_file)

['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['rh.inf.32k_fs_LR.surfnorm.func.gii']
['rh.inf.32k_fs_LR.coord.func.gii']
['rh.inf.32k_fs_LR.surfnorm.func.gii']
['rh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii', 'rh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii', 'rh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm.func.gii']
['lh.inf.32k_fs_LR.coord.func.gii']
['rh.inf.32k_fs_LR.surfnorm.func.gii']
['rh.inf.32k_fs_LR.coord.func.gii']
['lh.inf.32k_fs_LR.surfnorm